In [3]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/f5/d0/2e455d894ec0d6527e662ad55e70c04f421ad83a6fd0a54c3dd73c411282/opencv_python-4.8.0.76-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 3.5 MB/s eta 0:00:00m eta 0:00:010:00:01
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to su

In [ ]:
import moviepy.editor as mpy
import numpy as np
import cv2

def lip_sync(video_clip, audio_file):
    # Convert the audio file to a numpy array.
    audio_array = np.fromfile(audio_file, np.int16)

    # Convert the video clip to a list of frames.
    video_frames = [frame for frame in video_clip.iter_frames()]

    # Load and preprocess the lip mask.
    lip_mask = cv2.imread("lip_mask.png", cv2.IMREAD_GRAYSCALE)

    # Define the lip tracking region.
    lip_region = (x_start, y_start, x_end, y_end)  # Adjust these coordinates

    # Track the movement of the lips in the video clip.
    lip_positions = []
    for frame in video_frames:
        lip_frame = frame[y_start:y_end, x_start:x_end]
        lip_position = cv2.matchTemplate(lip_frame, lip_mask, cv2.TM_CCOEFF_NORMED)
        lip_positions.append(lip_position)

    # Create the output video clip with the original audio.
    lip_synced_clip = mpy.VideoFileClip("input.mp4").subclip(8, 10)
    lip_synced_clip = lip_synced_clip.set_audio(mpy.AudioFileClip(audio_file))

    # Calculate the lip position to frame mapping.
    lip_position_mapping = np.linspace(0, len(lip_positions) - 1, len(audio_array))

    # Adjust lip position values to fit the audio array length.
    scaled_lip_positions = np.interp(
        np.arange(len(audio_array)),
        np.arange(len(lip_positions)),
        lip_position_mapping
    )

    # Apply lip position adjustments to the video frames.
    def apply_lip_position(frame, lip_position):
        adjusted_frame = frame.copy()
        adjusted_frame[y_start:y_end, x_start:x_end] += lip_position
        return adjusted_frame

    lip_synced_frames = [
        apply_lip_position(frame, lip_positions[int(pos)])
        for pos, frame in zip(scaled_lip_positions, video_frames)
    ]

    # Create a video clip from the lip-synced frames.
    lip_synced_video_clip = mpy.ImageSequenceClip(lip_synced_frames, fps=video_clip.fps)

    # Save the resulting video clip.
    final_clip = mpy.concatenate_videoclips([video_clip, lip_synced_video_clip])
    final_clip.write_videofile("output.mp4", codec="libx264")

if __name__ == "__main__":
    video_clip = mpy.VideoFileClip("/home/fx/vikram_7-8/query/Change_Audio_Video/data/eng_clip.mp4")
    audio_file = "audio.wav"
    lip_sync(video_clip, audio_file)